In [4]:
import numpy as np 
from numpy import arctan2 as atan2, sin, cos
import matplotlib.pyplot as plt
from IPython import display
from scipy.constants import Boltzmann as kB 
from tkinter import *
from tkinter import ttk
from PIL import ImageGrab
import time


res = 700     # Resolution of the anymation 
tk = Tk()
tk.geometry( str(int(res*1.1)) + 'x'  +  str(int(res*1.3)) )
tk.configure(background='white')

canvas = Canvas(tk, bd=2)            # Generate animation window 
tk.attributes('-topmost', 0)
canvas.place(x=res/20, y=res/20, height= res, width= res)
ccolor = ['#17888E', '#E66C00', '#9E00C9', '#D80000', '#E87B00', '#9F68D3', '#4B934F']


dynamic_Vp = Scale(tk, from_=-100, to=100, orient=HORIZONTAL, label='Phoretic interaction strength')
dynamic_Vp.place(relx=.75, rely=.84, relheight= 0.12, relwidth= 0.2)
dynamic_Vp.set(0)


dynamic_T = Scale(tk, from_=0, to=100, orient=HORIZONTAL, label='Aligning interaction strength')
dynamic_T.place(relx=.5, rely=.84, relheight= 0.12, relwidth= 0.2)
dynamic_T.set(10)

dynamic_Dr = Scale(tk, from_=0, to=20, orient=HORIZONTAL, label='Rotational diffusion strength')
dynamic_Dr.place(relx=.27, rely=.84, relheight= 0.12, relwidth= 0.2)
dynamic_Dr.set(1)

dynamic_n = Scale(tk, from_=1, to=100, orient=HORIZONTAL, label='Number of particles')
dynamic_n.place(relx=.05, rely=.84, relheight= 0.12, relwidth= 0.2)
dynamic_n.set(20)



# Parameters of the simulation
na = dynamic_n.get()     # Number of particles 
n = 20
p_n = 350
N = 100000  # Simulation time
dt = 0.06   # Time step 

# Physical parameters of the system 
R = 4e-6      # Particle radius
l = 20*R        # Arena length
V = 10*R         # Particle velocity 
Vp = 20 * R  # Phoretic velocity amplitude
T0 = 10     # Aligning interactions amplitude
gamma = 6*np.pi*0.001*R
cD = np.sqrt(2*kB*300/gamma*dt) 
cDR = np.sqrt(2* kB*300/(8*np.pi*0.001*R**3) * dt)


x = np.random.rand(n)*2*l - l    # x coordinates            
y = np.random.rand(n)*2*l - l    # y coordinates  

px = np.random.rand(p_n)*2*l - l    # x coordinates            
py = np.random.rand(p_n)*2*l - l    # y coordinates  

phi = np.random.rand(n)*2*np.pi  # orientations                # Initialization 
nx = x                           # udpated x                   #       of 
ny = y                           # updated y                   #   parameters
T = np.zeros(n)                  # torque values
Fpx = np.zeros(n)                # phoretic attration forces along x
Fpy = np.zeros(n)                # phoretic attration forces along y

particles = []
p_particles = []
for j in range(n):     # Generate animated particles in Canvas 
    
    particles.append( canvas.create_oval( (x[j]-R + l)*res/l/2, \
                                          (y[j]-R + l)*res/2/l, \
                                          (x[j]+R + l)*res/l/2, \
                                          (y[j]+R + l)*res/l/2, \
                                          outline=ccolor[1], fill=ccolor[1]) )
for j in range(p_n): 
    p_particles.append( canvas.create_oval( (px[j]-R + l)*res/l/2, \
                                          (py[j]-R + l)*res/2/l, \
                                          (px[j]+R + l)*res/l/2, \
                                          (py[j]+R + l)*res/l/2, \
                                          outline=ccolor[0], fill=ccolor[0]) )
for i in range(N):
    Vp = dynamic_Vp.get()*R
    T0 = dynamic_T.get()
    Dr = dynamic_Dr.get()
    n  = dynamic_n.get()
    
    if n > len(x):
        x = np.append(x,  np.random.rand(n-len(T))*2*l - l)
        y = np.append(y,  np.random.rand(n-len(T))*2*l - l)
        
        phi = np.append(phi, np.random.rand(n-len(T))*2*np.pi)
        Fpx = np.append(Fpx, np.zeros(n-len(T)))
        Fpy = np.append(Fpy, np.zeros(n-len(T)))
        T = np.append(T, np.zeros(n-len(T)))
        for j in range(len(particles),n):
            particles.append( canvas.create_oval( (x[j]-R + l)*res/l/2, \
                                          (y[j]-R + l)*res/2/l, \
                                          (x[j]+R + l)*res/l/2, \
                                          (y[j]+R + l)*res/l/2, \
                                          outline=ccolor[1], fill=ccolor[1]) )
        
    if n < len(x):
        for j in range(n,len(x)):
            canvas.delete(particles[j])
        x = x[:n]
        y = y[:n]
        phi = phi[:n]
        particles = particles[:n]
        T = T[:n]
        Fpx = Fpx[:n]
        Fpy = Fpy[:n]  

    phi = phi + T*dt + Dr * cDR * np.random.randn(n)                                   # Update orientations
    nx = (x + Fpx*dt + V*cos(phi)*dt + cD * np.random.randn(n) + l) % (2*l) - l    # Update x coordinates
    ny = (y + Fpy*dt + V*sin(phi)*dt + cD * np.random.randn(n) + l) % (2*l) - l    # Update y coordinates
    
    p_nx = (px + cD * np.random.randn(p_n) + l) % (2*l) - l    # Update x coordinates
    p_ny = (py + cD * np.random.randn(p_n) + l) % (2*l) - l    # Update y coordinates
    
    
    
    clustering = np.zeros(n)
    for j in range(n):
        distances = np.sqrt((nx-nx[j])**2 + (ny-ny[j])**2)      # Calculate distances array to the particle       
        angles = atan2(ny-ny[j], nx-nx[j])                      # Directions of others array from the particle 
        clustering[distances < 2.5*R] = np.max( [np.max(clustering[distances<2.5*R]),\
                                                 np.sum(distances<2.5*R)-1])  # Clustering value of a particle 
        
        interact = distances < 10*R           # Create interaction indices 
        interact[j] = False                  # Do not interact with self

        Fpx[j] = np.sum( Vp/distances[interact]**2 * cos(angles[interact]) *(distances[interact]>2*R)) * R**2      #    Phoretic 
        Fpy[j] = np.sum( Vp/distances[interact]**2 * sin(angles[interact]) *(distances[interact]>2*R)) * R**2      #   attraction
        
        T[j] = - T0 * np.sum(cos(phi[j]-angles[interact])* \
                             sin(phi[j]-angles[interact])* \
                             R**2/distances[interact]**2)     #Aligning interactions
        
        
        distances = np.sqrt((p_nx-nx[j])**2 + (p_ny-ny[j])**2)      # Calculate distances array to the particle       
        angles = atan2(p_ny-ny[j], p_nx-nx[j])                      # Directions of others array from the particle 
        interact = distances < 10*R           # Create interaction indices 
        
        T[j] =  T0 * np.sum(cos(phi[j]-angles[interact])* \
                             sin(phi[j]-angles[interact])* \
                             R**2/distances[interact]**2)     #Aligning interactions
        
        
        
    for repeat in range(3):
        for j in range(p_n):
            distances = np.sqrt((p_nx-p_nx[j])**2 + (p_ny-p_ny[j])**2)      # Calculate distances array to the particle       
            angles = atan2(p_ny-p_ny[j], p_nx-p_nx[j])                      # Directions of others array from the particle 
            overlapp = distances < (2*R)                                                 #   Applying  
            overlapp[j] = False                                                          #    volume extraction
            for ind in np.where(overlapp)[0]:
                p_nx[j] = p_nx[j] + (distances[ind] - 2*R)*cos(angles[ind] )/2
                p_ny[j] = p_ny[j] + (distances[ind] - 2*R)*sin(angles[ind] )/2
                p_nx[ind] = p_nx[ind] - (distances[ind] - 2*R)*cos(angles[ind] )/2
                p_ny[ind] = p_ny[ind] - (distances[ind] - 2*R)*sin(angles[ind] )/2    
        
        
    for repeat in range(3):
        for j in range(n):
            distances = np.sqrt((nx-nx[j])**2 + (ny-ny[j])**2)      # Calculate distances array to the particle       
            angles = atan2(ny-ny[j], nx-nx[j])                      # Directions of others array from the particle 
            overlapp = distances < (2*R)                                                 #   Applying  
            overlapp[j] = False                                                          #    volume extraction
            for ind in np.where(overlapp)[0]:
                nx[j] = nx[j] + (distances[ind] - 2*R)*cos(angles[ind] )/2
                ny[j] = ny[j] + (distances[ind] - 2*R)*sin(angles[ind] )/2
                nx[ind] = nx[ind] - (distances[ind] - 2*R)*cos(angles[ind] )/2
                ny[ind] = ny[ind] - (distances[ind] - 2*R)*sin(angles[ind] )/2
                
            distances = np.sqrt((p_nx-nx[j])**2 + (p_ny-ny[j])**2)      # Calculate distances array to the particle       
            angles = atan2(p_ny-ny[j], p_nx-nx[j])                      # Directions of others array from the particle 
            overlapp = distances < (2*R)                                                 #   Applying  
                                                                                   #    volume extraction
            for ind in np.where(overlapp)[0]:
                nx[j] = nx[j] + (distances[ind] - 2*R)*cos(angles[ind] )/2
                ny[j] = ny[j] + (distances[ind] - 2*R)*sin(angles[ind] )/2
                p_nx[ind] = p_nx[ind] - (distances[ind] - 2*R)*cos(angles[ind] )/2
                p_ny[ind] = p_ny[ind] - (distances[ind] - 2*R)*sin(angles[ind] )/2
   
            


    clustering = np.min(np.array([clustering, np.ones(n)*6]),axis=0)               # maximum clustering 7       
    
    for j in range(n):
        canvas.move(particles[j], (nx[j]-x[j]) *res/l/2, (ny[j]-y[j])*res/l/2)     # Updating animation coordinates
    for j in range(p_n):
        canvas.move(p_particles[j], (p_nx[j]-px[j]) *res/l/2, (p_ny[j]-py[j])*res/l/2)     # Updating animation coordinates
        #canvas.itemconfig(particles[j], outline='#303030', fill=ccolor[int(clustering[j])]) #Colors 
    tk.title('t =' + str(round(i*dt*100)/100))          # Animation title 
    tk.update()                                         # Update animation frame 
    '''
    if i == 10:
        ImageGrab.grab().save('saveda.png')
    if i == 200:
        ImageGrab.grab().save('savedb.png')
    if i == 1000:
        ImageGrab.grab().save('savedc.png')
        '''
    time.sleep(0.001)                                   # Wait between loops
    x = nx                                              # Update x 
    y = ny                                              # Update y 
    
    px = p_nx                                              # Update x 
    py = p_ny                                              # Update y 
    
Tk.mainloop(canvas)                                     # Release animation handle (close window to finish)

TclError: invalid command name ".!scale"

50